In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 

In [2]:
t = tf.zeros([5,5,5,5])
t = tf.reshape(t, [625])
#print(t)

## Linear Regression Tutorial 

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt #visualization of charts and graphs
from IPython.display import clear_output
from six.moves import urllib 

import tensorflow.compat.v2.feature_column as fc #feature column for linear regression model 

import tensorflow as tf 

In [22]:
# Titanic Datasets Survival Rates
#dftrain.age.hist(bins=20)
#dftrain.sex.value_counts().plot(kind='barh') Gender bar graph sideways 
#dftrain['class'].value_counts().plot(kind='barh') Classes as side bargraph 
#pd.concat([dftrain, ytrain], axis= 1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')



dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

ytrain = dftrain.pop('survived') # Removed the survived column
y_eval = dfeval.pop('survived') # Removed the survived column


categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for feature_name in categorical_columns: 
    vocabulary = dftrain[feature_name].unique() # Gets a list of unique values 
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    
for feature_name in numeric_columns: 
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))


    
def make_input_fn(data_df, label_df, num_epocs=15, shuffle=True, batch_size=32): 
    def input_function(): 
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) #create a tf.data.dataset object with data and its la
        if shuffle: 
            ds = ds.shuffle(1000) # randomize order of data 
        ds = ds.batch(batch_size).repeat(num_epocs) # Split dataset into batches of 32 & repeat process
        return ds #return a batch of the dataset 
    return input_function # return a function object for use

train_input_fn = make_input_fn(dftrain, ytrain) #Call input function that was returned to use to get a a 
eval_input_fn = make_input_fn(dfeval, y_eval, num_epocs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn) # train
result = linear_est.evaluate(eval_input_fn) # get model metrics/stats by testing on testing data

clear_output() # clears console output
print(result) # the result variable is simply a dict of stats about our model 

{'accuracy': 0.75, 'accuracy_baseline': 0.625, 'auc': 0.8361799, 'auc_precision_recall': 0.7766994, 'average_loss': 0.47838226, 'label/mean': 0.375, 'loss': 0.46765268, 'precision': 0.6633663, 'prediction/mean': 0.37318197, 'recall': 0.67676765, 'global_step': 300}


In [37]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[2])
print('SURVIVED: ' + str(y_eval.loc[2]))
print(result[2]['probabilities'][1])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\jclaros\AppData\Local\Temp\tmp3mhpusoq\model.ckpt-300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                          58.0
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
SURVIVED: 1
0.63567215


# CLASSIFICATION TUTORIAL

In [86]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf 
import pandas as pd 

# Defining constants for model 
csv_column_names = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species'] 
species = ['Setosa', 'Versicolor', 'Virginica'] 

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=csv_column_names, header=0) 
test = pd.read_csv(test_path, names=csv_column_names, header=0) 


In [87]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [88]:
train_y = train.pop('Species') 
test_y = test.pop('Species') 
train.head() 

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [89]:
train.shape

(120, 4)

In [90]:
def input_fn(features, labels, training=True, batch_size=256): 
    #Convert the inputs to a dataset 
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle and repeat if you are in training mode 
    if training: 
        dataset = dataset.shuffle(1000).repeat() 
    
    return dataset.batch(batch_size)


my_feature_columns = [] 
for key in train.keys(): 
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None,

In [91]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each 
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectifully 
    hidden_units=[30, 10], 
    # The model must choose 3 classes 
    n_classes=3
    )

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), 
    steps=5000
    
)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jclaros\\AppData\\Local\\Temp\\tmpnz4fdj3c', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tenso

In [92]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-02-22T18:41:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\jclaros\AppData\Local\Temp\tmpnz4fdj3c\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28763s
INFO:tensorflow:Finished evaluation at 2024-02-22-18:41:05
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6666667, average_loss = 0.6333119, global_step = 5000, loss = 0.6333119
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\jclaros\AppData\Local\Temp\tmpnz4fdj3c\model.ckpt-5000

Test set accuracy: 0.667



In [94]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a dataset without labels 
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type Numeric values as prompted.") 

for feature in features: 
    valid = True 
    while valid:
        val = input(feature + ": ") 
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))

for pred_dict in predictions: 
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediciton is "{}" ({:.1f}%)'.format(Species[class_id], 100 * probability))

Please type Numeric values as prompted.
SepalLength: 2.4
SepalWidth: 2.6
PetalLength: 6.5
PetalWidth: 6.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\jclaros\AppData\Local\Temp\tmpnz4fdj3c\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


NameError: name 'Species' is not defined